In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

import lightgbm as lgb

from sklearn.metrics import r2_score, mean_squared_error
import pickle

In [0]:
MODEL_PATH = "/content/drive/Shared drives/CMPE-255/Trained Models"
DATASET2_PATH = "/content/drive/Shared drives/CMPE-255/final_df.csv"
FB_DATA_PATH = "/content/drive/Shared drives/CMPE-255/fb_status_data.csv"

In [7]:
df = pd.read_csv(DATASET2_PATH, encoding="ISO-8859-1")
X_df = df['status_update']
y_df = df.drop(['userid', 'status_update'], axis=1)
print(X_df.shape, y_df.shape)

(560237,) (560237, 5)


In [0]:
fb_status_dataset = pd.read_csv(FB_DATA_PATH, encoding="ISO-8859-1")
X_test = fb_status_dataset['STATUS']
drop_list = ['#AUTHID', 'STATUS', 'cEXT',
       'cNEU', 'cAGR', 'cCON', 'cOPN', 'DATE', 'NETWORKSIZE', 'BETWEENNESS',
       'NBETWEENNESS', 'DENSITY', 'BROKERAGE', 'NBROKERAGE', 'TRANSITIVITY']
#print(list_)
y_test = fb_status_dataset.drop(drop_list, axis=1)

In [0]:
#model 2: Light GBM
params = {
        'boosting' : 'dart',
        'application': 'regression',
        'max_depth': 10,
        'num_leaves': 100,
        'verbosity': -1,
        'metric': 'RMSE'}

es = lgb.early_stopping(stopping_rounds=10, verbose=True)
print_ev = lgb.print_evaluation(period=1, show_stdv=True)
#rate_decay = lgb.reset_parameter(learning_rate = lambda current_round: 0.01 * current_round)


In [0]:
traits = ['sOPN', 'sCON', 'sEXT', 'sAGR', 'sNEU']
y_pred = {}
y_pred_train = {}
tfidf = TfidfVectorizer(stop_words='english', strip_accents='ascii')
X = tfidf.fit_transform(X_df.values.astype('U'))
X_t = tfidf.transform(X_test.values.astype('U'))

In [11]:

pkl_file = '/content/drive/My Drive/255 Project Personality Prediction/pkl files/Copy of sOPN_model_lgbm.pkl'

with open(pkl_file, 'rb') as opn_pkl_model:
  lgbm_model = pickle.load(opn_pkl_model)
  opn_pkl_model.close()

print(lgbm_model)

In [0]:
output_path = '/content/drive/My Drive/255_pred_output'

personality_pred = lgbm_model.predict(X_t)

In [13]:
personality_pred[:10]


array([3.88121692, 3.88121692, 3.88121692, 3.88121692, 3.88121692,
       3.88121692, 3.88121692, 3.88121692, 3.88121692, 3.88121692])

In [0]:
personality_models = ['sCON', 'sEXT', 'sAGR', 'sNEU']

base_path = '/content/drive/My Drive/255 Project Personality Prediction/pkl files/'
model_name = 'lgbm'
preds = np.empty

for personality in personality_models:
  pkl_file = base_path + 'Copy of ' + personality + '_model_' + model_name + '.pkl'

  with open(pkl_file, 'rb') as opn_pkl_model:
    lgbm_model = pickle.load(opn_pkl_model)
    opn_pkl_model.close()
  
  preds = lgbm_model.predict(X_t)
  personality_pred = np.vstack((personality_pred, preds))

In [15]:
print(personality_pred.shape)

csv_pred = np.transpose(personality_pred)

print(csv_pred[:10,:])

(5, 9917)
[[3.88121692 3.43887588 3.60775124 3.55913213 2.78800717]
 [3.88121692 3.43887588 3.60775124 3.55913213 2.78800717]
 [3.88121692 3.43887588 3.60775124 3.55913213 2.78800717]
 [3.88121692 3.44698837 3.60775124 3.55913213 2.78800717]
 [3.88121692 3.43887588 3.60775124 3.55913213 2.78800717]
 [3.88121692 3.43887588 3.60775124 3.55913213 2.78800717]
 [3.88121692 3.45310289 3.60775124 3.55913213 2.78800717]
 [3.88121692 3.43887588 3.60775124 3.55913213 2.78800717]
 [3.88121692 3.43887588 3.60775124 3.55913213 2.78800717]
 [3.88121692 3.43887588 3.60775124 3.55913213 2.78800717]]


In [16]:
user_ids = fb_status_dataset['#AUTHID']
user_ids = user_ids.to_numpy()
print(type(user_ids))
user_ids = user_ids.reshape(user_ids.size, 1)
user_ids = np.hstack((user_ids, csv_pred))
# print(user_ids[:10])


<class 'numpy.ndarray'>


In [17]:
print(user_ids.shape)

(9917, 6)


In [0]:
users_df = pd.DataFrame(data=user_ids,columns=['UserIds', 'sOPN', 'sCON', 'sEXT', 'sAGR', 'sNEU'])


In [0]:
users_df.sOPN = pd.to_numeric(users_df.sOPN)

users_df.sCON = pd.to_numeric(users_df.sCON)
users_df.sEXT = pd.to_numeric(users_df.sEXT)
users_df.sAGR = pd.to_numeric(users_df.sAGR)
users_df.sNEU = pd.to_numeric(users_df.sNEU)

In [20]:
users_df.head()

,UserIds,sOPN,sCON,sEXT,sAGR,sNEU
0,b7b7764cfa1c523e4e93ab2a79a946c4,3.881217,3.438876,3.607751,3.559132,2.788007
1,b7b7764cfa1c523e4e93ab2a79a946c4,3.881217,3.438876,3.607751,3.559132,2.788007
2,b7b7764cfa1c523e4e93ab2a79a946c4,3.881217,3.438876,3.607751,3.559132,2.788007
3,b7b7764cfa1c523e4e93ab2a79a946c4,3.881217,3.446988,3.607751,3.559132,2.788007
4,b7b7764cfa1c523e4e93ab2a79a946c4,3.881217,3.438876,3.607751,3.559132,2.788007


In [23]:
print(users_df)

                               UserIds      sOPN  ...      sAGR      sNEU
0     b7b7764cfa1c523e4e93ab2a79a946c4  3.881217  ...  3.559132  2.788007
1     b7b7764cfa1c523e4e93ab2a79a946c4  3.881217  ...  3.559132  2.788007
2     b7b7764cfa1c523e4e93ab2a79a946c4  3.881217  ...  3.559132  2.788007
3     b7b7764cfa1c523e4e93ab2a79a946c4  3.881217  ...  3.559132  2.788007
4     b7b7764cfa1c523e4e93ab2a79a946c4  3.881217  ...  3.559132  2.788007
...                                ...       ...  ...       ...       ...
9912  deb899e426c1a5c66c24eeb0d7df6257  3.881217  ...  3.559132  2.788007
9913  ea28a927cb6663480ea33ca917c3c8ba  3.881217  ...  3.559132  2.788007
9914  ea28a927cb6663480ea33ca917c3c8ba  3.881217  ...  3.559132  2.788007
9915  5532642937eb3497a43e15dbb23a9d2d  3.881217  ...  3.559132  2.788007
9916  a286b7286b1247d4a7851709e9f31e1e  3.881217  ...  3.559132  2.788007

[9917 rows x 6 columns]


In [53]:
arr1 = users_df.to_numpy()
user_ids = arr1[:,:1]
user_score = np.array([2.5, 2.5, 2.5, 2.5, 2.5])
print(user_score)
cosine_sim = []
for row in arr1:
  comp_row = row[1:]
  sim = np.dot(user_score, comp_row)/ (np.linalg.norm(user_score) * np.linalg.norm(comp_row))
  
  cosine_sim.append(sim)
np_cosine = np.array(cosine_sim).reshape(-1,1)


sims = np.hstack((user_ids,np_cosine))
print(sims)



[2.5 2.5 2.5 2.5 2.5]
[['b7b7764cfa1c523e4e93ab2a79a946c4' 0.9945083618400787]
 ['b7b7764cfa1c523e4e93ab2a79a946c4' 0.9945083618400787]
 ['b7b7764cfa1c523e4e93ab2a79a946c4' 0.9945083618400787]
 ...
 ['ea28a927cb6663480ea33ca917c3c8ba' 0.9945083618400787]
 ['5532642937eb3497a43e15dbb23a9d2d' 0.9945083618400787]
 ['a286b7286b1247d4a7851709e9f31e1e' 0.9945083618400787]]


In [58]:
grouped_user_personalities = users_df.groupby('UserIds').mean()
grouped_user_personalities.head()

,sOPN,sCON,sEXT,sAGR,sNEU
UserIds,,,,,
00419a4c96b32cd63b2c7196da761274,3.881217,3.438876,3.607751,3.559132,2.788007
02c37028a782cfda660c7243e45244bb,3.881217,3.438876,3.607751,3.559132,2.788007
03133a828cd0cf52e3752813ce5d818f,3.881217,3.445366,3.607751,3.559132,2.788007
03e6c4eca4269c183fa0e1780f73faba,3.881217,3.434618,3.605389,3.559132,2.785771
06b055f8e2bca96496514891057913c3,3.881217,3.439129,3.607751,3.559132,2.788007


In [97]:
arr1 = grouped_user_personalities.reset_index().values
# print(arr1)
# # print(arrr.shape)
# # print(arr1[:,:1])
# i = 0
# for pref in arr1:
#   if i == 5:
#     break
#   i = i + 1
#   print(pref)

user_ids = arr1[:,:1]
user_score = np.array([0.0, 0.0, 0.9, 0.0, 0.0])

cosine_sim = []
for row in arr1:
  comp_row = row[1:]
  sim = np.dot(user_score, comp_row)/ (np.linalg.norm(user_score) * np.linalg.norm(comp_row))
  
  cosine_sim.append(sim)
np_cosine = np.array(cosine_sim).reshape(-1,1)


sims = np.hstack((arr1,np_cosine))
print(sims)


[['00419a4c96b32cd63b2c7196da761274' 3.8812169163216836
  3.4388758762390714 ... 3.5591321289967657 2.7880071693110913
  0.46442157720627797]
 ['02c37028a782cfda660c7243e45244bb' 3.8812169163216836
  3.4388758762390714 ... 3.5591321289967657 2.7880071693110913
  0.46442157720627797]
 ['03133a828cd0cf52e3752813ce5d818f' 3.8812169163216836 3.445365869480137
  ... 3.559132128996766 2.7880071693110913 0.46424974946357345]
 ...
 ['fc1c9fb6c64740edcbbf8cfe9dde8b02' 3.8812169163216805 3.440599873907399
  ... 3.5591321289967657 2.788007169311091 0.464636854834974]
 ['fe22087986fdcc65939c793fe0ec90a9' 3.8812169163216823
  3.4400524110881507 ... 3.559132128996766 2.7880071693110917
  0.4646029067811679]
 ['fffafe151f07a30a0ede2038a897b680' 3.881216916321682 3.443089188694897
  ... 3.559132128996766 2.7880071693110917 0.4643100415671519]]


In [98]:
sims_df = pd.DataFrame(data=sims,columns=['UserIds', 'sOPN', 'sCON', 'sEXT', 'sAGR', 'sNEU', 'similarity_scores'])
sims_df.head()

,UserIds,sOPN,sCON,sEXT,sAGR,sNEU,similarity_scores
0,00419a4c96b32cd63b2c7196da761274,3.88122,3.43888,3.60775,3.55913,2.78801,0.464422
1,02c37028a782cfda660c7243e45244bb,3.88122,3.43888,3.60775,3.55913,2.78801,0.464422
2,03133a828cd0cf52e3752813ce5d818f,3.88122,3.44537,3.60775,3.55913,2.78801,0.46425
3,03e6c4eca4269c183fa0e1780f73faba,3.88122,3.43462,3.60539,3.55913,2.78577,0.464344
4,06b055f8e2bca96496514891057913c3,3.88122,3.43913,3.60775,3.55913,2.78801,0.464415


In [0]:
sorted_sims_df = sims_df.sort_values(by=["similarity_scores"], ascending=False)


In [100]:
  top_ten = sorted_sims_df.iloc[:10]

  top_ten.to_numpy()

array([['6810431fc51108130be92d505ea4944b', 3.8812169163216805,
        3.4478035102749627, 3.6165415019299827, 3.5591321289967657,
        2.788007169311091, 0.4650717711026952],
       ['fc1c9fb6c64740edcbbf8cfe9dde8b02', 3.8812169163216805,
        3.440599873907399, 3.610336141432862, 3.5591321289967657,
        2.788007169311091, 0.464636854834974],
       ['325e62f4e7e4f64a03fcf831a8d80bf1', 3.8812169163216836,
        3.4388758762390714, 3.609790991619162, 3.5591321289967657,
        2.7880071693110913, 0.4646274800909505],
       ['530e195e467f673de0917893fbf9299a', 3.8812169163216805,
        3.4406174675352337, 3.6102552643777637, 3.559879910922718,
        2.787169990487735, 0.464625703131461],
       ['b4de950df13c0765018013237c6a1b3d', 3.8812169163216836,
        3.4414164945308037, 3.610774129052596, 3.5624971476635534,
        2.7856691658591632, 0.4646173682115088],
       ['450c787001b004af69428e267c7a4ca1', 3.881216916321679,
        3.440926106347942, 3.6097885982163

In [0]:
grouped_user_personalities.to_csv('/content/drive/My Drive/255_pred_output/personality_predictions.csv')